In [1]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import os
import time

In [2]:
# Указываем путь к файлу
path = 'C:\\Users\home\\Downloads\\Ефремов_-_Лезвие_бритвы.txt'

In [3]:
# Открываем файл на чтение
text = open(path, 'r').read()

In [4]:
# Печатаем длину строки в символах
print('Длина текста: {} символов.'.format(len(text)))

Длина текста: 1316322 символов.


In [5]:
# Печатаем первые 250 символов
print(text[:250])



Иван Ефремов

Лезвие бритвы




Пролог


Все быстрее нарастает познание в современном мире. Обрисовывается точнейшая взаимосвязь, обусловленность кажущихся различными явлений мира и жизни. Всеобщее переплетение отдаленных случайностей, вырастающее 


In [6]:
# Получаем уникальные символы в строке
vocab = sorted(set(text))
print('{} уникальных символа'.format(len(vocab)))

114 уникальных символа


In [7]:
# Векторизуем текст. Получаем числовое представление каждого слова
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
# Выводим числовое представление для первых 20 символов
print('{')
for char, _ in zip(char2idx, range(20)):
    print('    {:4s}: {:3d}'.format(repr(char), char2idx[char]))
print('   ...\n}')

{
    '\n':   0
    ' ' :   1
    '!' :   2
    '(' :   3
    ')' :   4
    ',' :   5
    '-' :   6
    '.' :   7
    '/' :   8
    '0' :   9
    '1' :  10
    '2' :  11
    '3' :  12
    '4' :  13
    '5' :  14
    '6' :  15
    '7' :  16
    '8' :  17
    '9' :  18
    ':' :  19
   ...
}


In [9]:
# Выводим первые 20 символов текста, представленные в виде чисел
print('{} ---- символов преобразовано в числа ----> {}'.format(repr(text[:20]), text_as_int[:13]))

'\n\nИван Ефремов\n\nЛезв' ---- символов преобразовано в числа ----> [ 0  0 56 79 77 90  1 53 97 93 82 89 91]


In [10]:
# Максимальная длина предложения необходима для одного ввода символов 
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Создаем тренировочные примеры и цели
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(20):
    print(idx2char[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.




И
в
а
н
 
Е
ф
р
е
м
о
в




Л
е
з
в


In [11]:
# Выводим текст строками (типа предложениями по 100 символов)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(20):
    print(repr(''.join(idx2char[item.numpy()])))

'\n\nИван Ефремов\n\nЛезвие бритвы\n\n\n\n\nПролог\n\n\nВсе быстрее нарастает познание в современном мире. Обрисов'
'ывается точнейшая взаимосвязь, обусловленность кажущихся различными явлений мира и жизни. Всеобщее пе'
'реплетение отдаленных случайностей, вырастающее в необходимость, то есть в законы природы, пожалуй, с'
'амое важное прозрение современного человека.\n\nИ в человеческом существовании незаметные совпадения, д'
'авно наметившиеся сцепления обстоятельств, тонкие нити, соединяющие те или другие случайности, выраст'
'ают в накрепко спаянную логическую цепь, влекущую за собой попавшие в ее орбиту человеческие жизни. М'
'ы, не зная достаточно глубоко причинную связь, не понимая истинных мотивов, называем это судьбой.\n\nЕс'
'ли проследить всю цепь, а затем распутать начальные ее нити, можно прийти к некоему отправному момент'
'у, послужившему как бы спусковым крючком или замыкающей кнопкой. Отсюда начинается долгий ряд событий'
', неизбежно долженствующих сблизить совершенно 

In [12]:
# Создаем функцию, возвращающую каждое предложение в форме для ввода
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
# Печатаем первые примеры
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n\nИван Ефремов\n\nЛезвие бритвы\n\n\n\n\nПролог\n\n\nВсе быстрее нарастает познание в современном мире. Обрисо'
Target data:  '\nИван Ефремов\n\nЛезвие бритвы\n\n\n\n\nПролог\n\n\nВсе быстрее нарастает познание в современном мире. Обрисов'


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print('Step {:4d}'.format(i))
    print('    input: {} ({:s})'.format(input_idx, repr(idx2char[input_idx])))
    print('    expected output: {} ({:2})'.format(input_idx, repr(idx2char[target_idx])))

Step    0
    input: 0 ('\n')
    expected output: 0 ('\n')
Step    1
    input: 0 ('\n')
    expected output: 0 ('И')
Step    2
    input: 56 ('И')
    expected output: 56 ('в')
Step    3
    input: 79 ('в')
    expected output: 79 ('а')
Step    4
    input: 77 ('а')
    expected output: 77 ('н')


In [15]:
# Создаем тренировочные пачки
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [16]:
# Строим модель
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [17]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [18]:
# Создаем функцию, которая возвращает модель
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
           return_sequences=True,
           recurrent_initializer='glorot_uniform',
           stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
# Инициализируем модель
model = build_model(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [20]:
# Испытываем модель
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, '# (batch_size, sequence_length, vocab_size)')

(64, 100, 114) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           29184     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 114)           116850    
Total params: 4,081,266
Trainable params: 4,081,266
Non-trainable params: 0
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [25]:
sampled_indices

array([ 91,  86,   7,  55,  15,  40,  15, 110,  42,  47,  55, 108, 107,
        89,   2,  86,  65,  94,  22,  50,  51,  55,  13,  37, 110,   8,
        19,  54,  88,  48,  75,  63,  31,   0,  88,  49,  28,  27, 101,
        10, 113,  33,  90,  43,  98,  31, 101,   8,  87, 107, 100,  79,
        48, 107,  77,  45,   9, 102,   1,  71,  28,  79,  80,  16,  51,
       110, 113,  72,  76,  59, 108,  83,  57,  38,  28,  16,  97,   2,
        33,  31,  45, 105,  98,  99,  74,  32,  25,  11,  31,  10,  32,
        62,   5,  36,  20,  51,  92,  60, 103,  36], dtype=int64)

In [26]:
# Выводим предсказание нетренированной модели
print('input: \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Next char Predictions: \n', repr(''.join(idx2char[sampled_indices ])))

input: 
 'росила Сима.\n\n–\xa0Вероятно, в дальнейшем наука будет способна и не на такие чудеса. Действительно, все'

Next char Predictions: 
 'ой.З6r6–t»Зяюм!йСсIВГЗ4n–/:ЖлАЮПg\nлБcbш1…iнuхgш/кючвАюа\xa00щ Чcвг7Г–…ШЯЛяжЙoc7ф!ig\xa0ьхцЭhX2g1hО,m;ГпМъm'


In [28]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print('Predication shape: ', example_batch_predictions.shape, ' # (batch_size, sequence_length, vocab_size)')
print('Scalar_loss:       ', example_batch_loss.numpy().mean())

Predication shape:  (64, 100, 114)  # (batch_size, sequence_length, vocab_size)
Scalar_loss:        4.7374616


In [30]:
# Настраиваем тренировочную процедуру
model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
# Выполняем тренировку
EPOCHS = 3
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
204/205 [============================>.] - ETA: 1:29 - loss: 2.7981WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
205/205 [==============================] - 18438s 90s/step - loss: 2.7958
Epoch 2/3
205/205 [==============================] - 19863s 97s/step - loss: 2.1675
Epoch 3/3
205/205 [==============================] - 14697s 72s/step - loss: 1.8855


In [34]:
# Генерируем текст
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            29184     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 114)            116850    
Total params: 4,081,266
Trainable params: 4,081,266
Non-trainable params: 0
_________________________________________________________________


In [36]:
def generate_text(model, start_string):
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))
print(generate_text(model, start_string=u'Наука '))

Наука – верно, что все дочь имоевственный в десторнам, – уловья – не протрогие задерев в эте. Чида может желаной, сефень и слота дваю женщина. А пот экрулих многом, хучались необочить в орином. Гирин цердал, что онг Гирин и бящую Кимства. Якотоло этой только заделжавший цеания рукном и давнах целую! Наустал своягдушим, сейдне, Чезаре в слученную серятнице начу! – Гирин рас встовор, Нараверно? но девяшка, новикие обказание в Востою время. Чазаре. А тенного зверьяться в хаш бы пока, тло сила, собывойсонаят к яды и паттиступа постагоря. Чакая мужчанам, судала Леа.

– Пойте понимаются в корОне для шехга. Умерносят крокроги двиют этах буды, невироков, что вы длянные ли.. На Аласкапльно простат тержение кону, как аякапальная охтати оставать путь часов. – Ризамисто и дюмак хидушнов Челезик… Блазым вас Гирина устудного, прозык, порожая разговой. И время эта тели, – Может пей, которые слобость? – осторожно был у на» в образрении. Сенчас сойсне этскудать взгляд.

Сонаренно! – работа падральству 